In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
import re

from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse

In [ ]:
train=pd.read_csv('/kaggle/input/weekend-hackathon-edition-2-sentiment/train.csv')
test=pd.read_csv('/kaggle/input/weekend-hackathon-edition-2-sentiment/test.csv')
sample_sub = pd.read_csv('/kaggle/input/weekend-hackathon-edition-2-sentiment/submission.csv')

In [ ]:
sample_sub.columns

In [ ]:
train.head()

In [ ]:
def preprocess_reviews(text):
    text = re.sub(r'<[^>]*>', ' ', text, re.UNICODE)
    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    text = re.sub(r'[^0-9a-zA-Z]+',' ',text, re.UNICODE)
    text = " ".join(text.split())
    text = text.lower()
    return text

train['Review'] = train.Review.apply(lambda x: preprocess_reviews(x))
test['Review'] = test.Review.apply(lambda x: preprocess_reviews(x))

In [ ]:
train.head()

In [ ]:
#del train['ID']
#del train
#test_id=test[['ID']]
#del test['ID']
print(train.shape)
del train['ID']
del train['author']

train.drop_duplicates(keep= 'first',inplace=True)
train=train.reset_index(drop=True)
print(train.shape)

In [ ]:
train['Review'].apply(lambda x: len(x)).describe()

In [ ]:
def get_model(model_type, model_name, n_epochs = 5, train_batch_size = 16, eval_batch_size = 16, seq_len = 64, lr = 2e-5):
  model = ClassificationModel(model_type, model_name,num_labels=3, args={'train_batch_size':train_batch_size,
                                                                         "eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': n_epochs,
                                                                         'max_seq_length': seq_len,
                                                                         'manual_seed': 12345,
                                                                         "learning_rate":lr,
                                                                         "n_gpu": 8,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,})
  return model

In [ ]:
tmp = pd.DataFrame()
tmp['text'] = train['Review']
tmp['labels'] = train['Sentiment']
tmp_test = test[['Review']].rename({'Review': 'text'}, axis=1)
tmp_test['labels'] = 0
tmp_trn, tmp_val = train_test_split(tmp, test_size=0.3, random_state=12345)

In [ ]:
model = get_model('roberta', 'roberta-base', n_epochs=5)
model.train_model(tmp)
#preds_val = model.eval_model(tmp_val)[1]


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(tmp_test)

In [ ]:
from scipy.special import softmax

probabilities = softmax(model_outputs, axis=1)
probabilities

In [ ]:
out_pb = pd.DataFrame(probabilities)
out_pb.columns=['Negative_0', 'Neutral_1', 'Positive_2']
out_pb.to_csv('/kaggle/working/out_pb_roberta_base_64_16_5.csv',index=False)

With CV:

In [ ]:
from sklearn.model_selection import KFold, RepeatedKFold
fold = KFold(n_splits=3, shuffle=True, random_state=12345)
y_pred_totcb = []
i=1
for train_index, test_index in fold.split(tmp):
    train_df = tmp.iloc[train_index]
    valid_df = tmp.iloc[test_index]
    
    
    
    model = get_model('roberta', 'roberta-base', n_epochs=5)
    model.train_model(train_df)
    
    result, model_outputs, wrong_predictions = model.eval_model(test)
    y_pred_totcb.append(model_outputs)
    print(f'fold {i} completed')
    i = i+1
    
    
result=np.mean(y_pred_totcb, axis=0)
from scipy.special import softmax
out_pb=softmax(result, axis=1)
out_pb = pd.DataFrame(out_pb)

out_pb.columns=['Negative_0', 'Neutral_1', 'Positive_2']
out_pb.to_csv('/kaggle/working/out_pb_roberta_base_64_16_5_CV.csv',index=False)